> ## Neste notebook o usuário deve entrar com o nome da lei, o inciso e a alínea, e o programa retornará cada um dos parâmetros solicitados.

In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import json

In [ ]:
url = 'https://www2.camara.leg.br/busca/?q=lei 9503'
option = Options()
option.headless = True
driver = webdriver.Firefox(options=option)
driver.get(url)
time.sleep(10)
driver.quit()

In [3]:
#simulando o click
url = 'https://www2.camara.leg.br/busca/?q=lei 9503'
option = Options()
option.headless = True
driver = webdriver.Firefox()
driver.get(url)
time.sleep(10)
driver.find_element_by_xpath("//div[@id='resultadoBusca']//ul//li//span//a").click()
time.sleep(10)
driver.find_element_by_xpath("//div[@id='resultadoBusca']//ul//li//span//a").click()
"""
//div[@class='nba-stat-table']//table//thead//tr//th[@data-field='PTS'] é o caminho que o navegador vai fazer
para dar o click
"""
#pegando a tabela
#element = driver.find_element_by_xpath("//div[@class='nba-stat-table']//table")
#html_content = element.get_attribute('outerHTML')
#print(html_content)
#driver.quit()

"\n//div[@class='nba-stat-table']//table//thead//tr//th[@data-field='PTS'] é o caminho que o navegador vai fazer\npara dar o click\n"